# Classifying Text

We will use the "RottenTomatoes" dataset and classify the review texts for sentiment. We will classify the test split of the dataset and evaluate the results of the classifier against the true labels in the test split of the dataset.

We will use the pipeline module from the transformers package

In [ ]:
#!pip install evaluate

Import Libraries

In [5]:
from datasets import load_dataset
from evaluate import evaluator, combine
from transformers import pipeline
import torch

load the RottenTomatoes dataset and select the first five sentences from it.

In [6]:
sentences = load_dataset(
    "rotten_tomatoes",
    split = "test"
).select(range(5))

[print(sentence) for sentence in sentences["text"]]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
consistently clever and suspenseful .
it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .
the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .
red dragon " never cuts corners .


[None, None, None, None, None]

Initialize the pipeline for sentiment analysis via a pipeline.

A pipeline is an abstraction that allows us to easily use models or inference tasks without having to write the code to piece them together.

In [7]:
roberta_pipe = pipeline("sentiment-analysis",
                        model = "textattack/roberta-base-rotten-tomatoes")

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-rotten-tomatoes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


Generate the predictions for the small subset of sentences that we selected in step 2.

In [9]:
predictions = roberta_pipe(list(sentences["text"]))

We iterate through our sentences and check the predictions for our sentences. We print the actual and generated predictions along with the sentence text for the five sentences.

In [10]:
for idx, _sentence in enumerate(sentences["text"]):
  print(
      f"actual: {sentences["label"][idx]}\n"
      f"predicted: {"1" if predictions[idx]["label"] == "LABEL_1" else "0"}\n"
      f"sentence: {_sentence}\n\n"
  )

actual: 1
predicted: 1
sentence: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


actual: 1
predicted: 1
sentence: consistently clever and suspenseful .


actual: 1
predicted: 0
sentence: it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .


actual: 1
predicted: 1
sentence: the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .


actual: 1
predicted: 1
sentence: red dragon " never cuts corners .




Now that we have validated the pipeline and its results, let’s generate the inference for the whole test set and generate the evaluation measures of this particular model. Load the complete test split for the RottenTomatoes dataset:

In [11]:
sentences = load_dataset("rotten_tomatoes",
                         split = "test")

We initialize an evaluator object that can be used to perform the inference along with evaluating the results of the classification. It can also be used to present an easy-to-read summary of the evaluation results:

In [12]:
task_evaluator = evaluator("sentiment-analysis")

We call the compute method on the evaluator instance. This triggers the inference and the evaluation using the same pipeline instance that we initialized in step 4. It returns the evaluation metrics of accuracy, precision, recall, and f1, along with some performance metrics related to inference:

In [13]:
eval_results = task_evaluator.compute(
    model_or_pipeline = roberta_pipe,
    data = sentences,
    metric = combine(["accuracy",
                      "precision", "recall", "f1"]),
    label_mapping = {"LABEL_0": 0, "LABEL_1":1}
)

print the results of the evaluation

In [14]:
print(eval_results)

{'accuracy': 0.8874296435272045, 'precision': 0.9222903885480572, 'recall': 0.8461538461538461, 'f1': 0.8825831702544031, 'total_time_in_seconds': 204.18769715800045, 'samples_per_second': 5.220686725190545, 'latency_in_seconds': 0.19154568213696102}
